## Import & 전역변수

In [20]:
import requests
import json
TOT_CNT = 157116
MAX_PER_PAGE = 100
ITER_NUM = TOT_CNT//MAX_PER_PAGE+1
CHECK_POINT = 300

## 영화진흥위원회 API를 사용하여 영화목록 가져오기


In [10]:
URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
for i in range(ITER_NUM):
    if i != 0 and i%CHECK_POINT == 0 :
        print(f"Now : {i}")
    PAGE_NUM = i+1
    params = {
        "key" : "74c13a6087207fc051ab22dd0752be12",
        "curPage" : PAGE_NUM,
        "itemPerPage" : MAX_PER_PAGE,
    }
    response = requests.get(URL, params=params)
    # print(response.json())
    # break
    with open(f'./json/people_list_{PAGE_NUM}.json', 'w') as f :
        json.dump(response.json(), f)

Now : 300
Now : 600
Now : 900
Now : 1200
Now : 1500


## API를 이용해 저장한 데이터 불러오기


In [16]:
TOTAL_DATA_COUNT = 0
for i in range(ITER_NUM) :
    if i != 0 and  i % CHECK_POINT == 0 :
        print(f"Now : {i}")
    with open(f'./json/people_list_{i+1}.json', 'r') as f :
        content = json.load(f)
        peopleListResult = content['peopleListResult']
        TOTAL_COUNT = peopleListResult['totCnt']
        PEOPLE_LIST = peopleListResult['peopleList']
        TOTAL_DATA_COUNT  += len(PEOPLE_LIST)

print(f"DB에 저장된 영화의 갯수 : {TOTAL_COUNT}")
print(f"현재 API로 불러 온 영화의 갯수 : {TOTAL_DATA_COUNT}")

Now : 301
Now : 601
Now : 901
Now : 1201
Now : 1501
DB에 저장된 영화의 갯수 : 157116
현재 API로 불러 온 영화의 갯수 : 157116


## API로 불러온 컬럼 분석

In [17]:
NUM_LEN_ONE_COL = set()
NUM_LEN_OTHER_COL = set()

for i in range(ITER_NUM):
    if i != 0 and  i % CHECK_POINT == 0 :
        print(f"Now : {i}")
    with open(f'./json/people_list_{i+1}.json', 'r') as f :
        content = json.load(f)
        peopleListResult = content['peopleListResult']
        PEOPLE_LIST = peopleListResult['peopleList']
        for PEOPLE in PEOPLE_LIST :
            for key, val in PEOPLE.items() :
                if type(val) != list :
                    NUM_LEN_ONE_COL.add(key)
                else :
                    NUM_LEN_OTHER_COL.add(key)

print(f"len(ONE_COL) : {len(NUM_LEN_ONE_COL)}, ONE_COL : {NUM_LEN_ONE_COL}")
print(f"len(OTHER_COL) : {len(NUM_LEN_OTHER_COL)}, OTHER_COL : {NUM_LEN_OTHER_COL}")

Now : 301
Now : 601
Now : 901
Now : 1201
Now : 1501
len(ONE_COL) : 5, ONE_COL : {'peopleCd', 'filmoNames', 'repRoleNm', 'peopleNm', 'peopleNmEn'}
len(OTHER_COL) : 0, OTHER_COL : set()


## Table 만들기

In [18]:
result = {}
for i in range(ITER_NUM):
    if i % CHECK_POINT == 0 :
        print(f"Now : {i}")
    with open(f'./json/people_list_{i+1}.json', 'r') as f :
        content = json.load(f)
        peopleListResult = content['peopleListResult']
        PEOPLE_LIST = peopleListResult['peopleList']
    for PEOPLE in PEOPLE_LIST :
        people_code = None
        for key, val in PEOPLE.items():
            if type(val) == list :
                continue
            if key == 'peopleCd' :
                result[val] = dict()
                people_code = val
            try :
                result[people_code][key] = val
            except KeyError :
                continue

print(f"Num of people : {len(result)}")


Now : 0
Now : 300
Now : 600
Now : 900
Now : 1200
Now : 1500
Num of people : 157116


## Pickle, Excel 파일로 저장하기

In [19]:
import pickle
with open("people.pickle", 'wb') as bf:
    pickle.dump(result, bf)

import pandas as pd
df = pd.DataFrame.from_dict(result).T
with pd.ExcelWriter("./excel/people.xlsx", mode='w') as W:
    df.to_excel(W)